Todo 

- [ ] Checar dados faltantes em raca e sexo

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Datasets
url_discentes_2019 = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/a55aef81-e094-4267-8643-f283524e3dd7/download/discentes-2019.csv"
url_cursos = "http://dados.ufrn.br/dataset/02526b96-cf40-4507-90b0-3afe5ddd53e7/resource/a10bc434-9a2d-491a-ae8c-41cf643c35bc/download/cursos-de-graduacao.csv"

# ColunaS
COLUNAS_ALUNO = ['sexo', 'raca', 'tipo_discente', 'status', 'sigla_nivel_ensino', 'matricula', 'id_curso']
COLUNAS_CURSO = ['nome', 'nivel_ensino', 'area_conhecimento', 'id_curso']

discentes = pd.read_csv(url_discentes_2019, sep=';')
alunos_ativos_2019 = (discentes.status == 'ATIVO') & (discentes.sigla_nivel_ensino == 'G')
dados_discentes = discentes.loc[ alunos_ativos_2019 , COLUNAS_ALUNO ]

cursos = pd.read_csv(url_cursos, sep=';')
cursos_graduacao = cursos.loc[ cursos.nivel_ensino == 'GRADUAÇÃO',  COLUNAS_CURSO ]
dist_cursos_2019 = cursos_graduacao.set_index('id_curso')

### Pre processamento

In [9]:
cursos_graduacao[['nome','area_conhecimento']].drop_duplicates()['nome'].value_counts()

GEOGRAFIA                                2
PEDAGOGIA                                2
PSICOLOGIA                               2
LETRAS - LÍNGUA PORTUGUESA               2
QUÍMICA DO PETRÓLEO                      1
FÍSICA                                   1
COMUNICAÇÃO SOCIAL - AUDIOVISUAL         1
ESTATÍSTICA                              1
ENGENHARIA MECATRÔNICA                   1
METEOROLOGIA                             1
ENGENHARIA DE PETRÓLEO                   1
ECOLOGIA                                 1
FISIOTERAPIA                             1
LETRAS                                   1
ENGENHARIA DE COMPUTAÇÃO                 1
BIOMEDICINA                              1
FONOAUDIOLOGIA                           1
ENGENHARIA AMBIENTAL                     1
ENFERMAGEM                               1
ANÁLISE E DESENVOLVIMENTO DE SISTEMAS    1
ENGENHARIA DE ALIMENTOS                  1
ENGENHARIA AGRONÔMICA                    1
ENGENHARIA DE SOFTWARE                   1
CIÊNCIAS AT

In [10]:
cursos_graduacao.head()

,nome,nivel_ensino,area_conhecimento,id_curso
0,ADMINISTRAÇÃO,GRADUAÇÃO,Ciências Sociais Aplicadas,2000002
1,ADMINISTRAÇÃO PÚBLICA,GRADUAÇÃO,Ciências Sociais Aplicadas,15315770
2,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS,GRADUAÇÃO,Outra,94598200
3,ARQUITETURA E URBANISMO,GRADUAÇÃO,Engenharias,2000005
4,ARTES VISUAIS,GRADUAÇÃO,"Linguística, Letras e Artes",2000123


## Como está hoje?

Vamos começar com a porcentagem de mulheres por curso

In [3]:
# porcentagem de mulheres por curso
cursos_e_alunos = pd.merge(left=cursos_graduacao, right=dados_discentes, on='id_curso')

In [4]:
cursos_e_alunos.columns

Index(['nome', 'nivel_ensino', 'area_conhecimento', 'id_curso', 'sexo', 'raca',
       'tipo_discente', 'status', 'sigla_nivel_ensino', 'matricula'],
      dtype='object')

In [5]:
cursos_sexos_count = pd.pivot_table(cursos_e_alunos, values='matricula', index=['id_curso'],                    
            columns=['sexo'], aggfunc=lambda x: len(x), fill_value=0)

In [6]:
cursos_sexos_count.head()

sexo,F,M
id_curso,,
2000002,37,73
2000005,13,8
2000006,23,15
2000009,13,9
2000011,37,56


In [7]:
# Calcula a porcentagem de mulheres por curso
dist_cursos_2019['mulheres_no_curso(%)']= cursos_sexos_count['F'] / (cursos_sexos_count['F'] + cursos_sexos_count['M'])

In [8]:
print(dist_cursos_2019['mulheres_no_curso(%)'].head())

id_curso
2000002     0.336364
15315770    0.000000
94598200    0.214286
2000005     0.619048
2000123     0.545455
Name: mulheres_no_curso(%), dtype: float64


In [9]:
cursos_racas_count = pd.pivot_table(cursos_e_alunos, values='matricula', index=['id_curso'],                    
            columns=['raca'], aggfunc=lambda x: len(x), fill_value=0)

cursos_racas_count.columns = cursos_racas_count.columns.str.lower()

names = {
    "amarelo (de origem oriental)": "amarelo",
    "indígeno": "indigeno",
    "não informado": "nao_informado"
}
cursos_racas_count = cursos_racas_count.rename(columns=names)

In [10]:
cursos_racas_count.head()

raca,amarelo,branco,indigeno,negro,nao_informado,pardo
id_curso,,,,,,
2000002,1,54,1,10,1,43
2000005,0,13,0,1,0,7
2000006,0,17,0,4,3,14
2000009,0,10,0,3,1,8
2000011,0,42,0,3,1,47


In [11]:
total_racas = cursos_racas_count.sum(axis='columns')

for raca in cursos_racas_count.columns:
  dist_cursos_2019[raca+'_no_curso(%)']= cursos_racas_count[raca] / total_racas

In [12]:
dist_cursos_2019.head()

,nome,nivel_ensino,area_conhecimento,mulheres_no_curso(%),amarelo_no_curso(%),branco_no_curso(%),indigeno_no_curso(%),negro_no_curso(%),nao_informado_no_curso(%),pardo_no_curso(%)
id_curso,,,,,,,,,,
2000002,ADMINISTRAÇÃO,GRADUAÇÃO,Ciências Sociais Aplicadas,0.336364,0.009091,0.490909,0.009091,0.090909,0.009091,0.390909
15315770,ADMINISTRAÇÃO PÚBLICA,GRADUAÇÃO,Ciências Sociais Aplicadas,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
94598200,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS,GRADUAÇÃO,Outra,0.214286,0.000000,0.285714,0.000000,0.047619,0.071429,0.595238
2000005,ARQUITETURA E URBANISMO,GRADUAÇÃO,Engenharias,0.619048,0.000000,0.619048,0.000000,0.047619,0.000000,0.333333
2000123,ARTES VISUAIS,GRADUAÇÃO,"Linguística, Letras e Artes",0.545455,0.022727,0.522727,0.000000,0.090909,0.000000,0.363636


In [13]:
dados_discentes.loc[ dados_discentes.id_curso == 15315770]

,sexo,raca,tipo_discente,status,sigla_nivel_ensino,matricula,id_curso
10301,M,Branco,REGULAR,ATIVO,G,20190000026,15315770.0


Agora vamos plotar os gráficos usando a biblioteca Bokeh

In [16]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [25]:
# Make a list of the unique values from the region column: regions_list
areas_list = dist_cursos_2019.area_conhecimento.unique().tolist()

# Import CategoricalColorMapper from bokeh.models and the Spectral6 palette from bokeh.palettes
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6

source = ColumnDataSource(data={
    'x'       : dist_cursos_2019["pardo_no_curso(%)"],
    'y'       : dist_cursos_2019["mulheres_no_curso(%)"],
    'curso'   : dist_cursos_2019.nome,
    'area'    : dist_cursos_2019.area_conhecimento,
})

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=areas_list, palette=Spectral6)

# Create the figure: plot
plot = figure(title='RELAÇÃO ENTRE RAÇA/COR E GÊNERO NOS CURSOS', plot_height = 600, plot_width = 800)


# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', fill_alpha=0.8, source=source,
            color=dict(field='area', transform=color_mapper), legend='area',
           size=10)

# Set the x-axis label
plot.xaxis.axis_label ='Porcantagem de alunos pardos no curso'

# Set the y-axis label
plot.yaxis.axis_label = 'Porcentagem de mulheres no curso'

# Set the legend.location attribute of the plot to 'top_right'
plot.legend.location = 'bottom_right'

# Add the plot to the current document and add the title
curdoc().add_root(plot)
curdoc().title = 'Gapminder'

In [26]:
show(plot)

## Como era antes?

## Como estamos em relação ao Brasil?